## Sentimenet Analysis with fastai

In [ ]:
!pip install -U fastai

In [ ]:
import pandas as pd
from fastai.text.all import *

In [ ]:
raw_df = pd.read_excel("./drive/MyDrive/digikala.xlsx")

In [ ]:
raw_df.columns

Index(['product_id', 'product_title', 'title_en', 'user_id', 'likes',
       'dislikes', 'verification_status', 'recommend', 'title', 'comment',
       'advantages', 'disadvantages'],
      dtype='object')

In [ ]:
df = raw_df.copy()
df["text"] = df["title"].astype(str) + "\r\n" + df["comment"].astype(str)

In [ ]:
df = df[df.recommend.isin(["recommended", "not_recommended"])].loc[:, ["text", "recommend"]]

### Specific text classification task with pretrained model


In [ ]:
dls = TextDataLoaders.from_df(df, text_col="text", label_col="recommend", valid_pct=0.1)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
dls.show_batch()

,text,category
0,"xxbos بررسی sandisk xxmaj ultra xxmaj fit xxmaj flash 128 gb \r\n می بخشید طولانیه اما امیدوارم مفید باشه براتون . \r\n قبل هر چیزی در مورد نظرات یه مطلب بگم ، اینکه بعضی دوستان گفتن خوبه و سرعتش بالاست و بعضی دوستان هم گفتن بد و خیلی سرعتش پایین … در هر دو حالات هر xxunk درست گفتن ! نکته اینجاست ، کسی که میاد و نظرش میگه مشخصات سخت افزاری xxunk ( لب تاپ یا کیس و .. ) رو نمیگه که خیلی خیلی مهمه ! مثلا سرعت خواندن و نوشتن هاردش 5400 دوره یا xxunk دوره ، ( البته سرعت یک هارد xxunk فیک ممکنه از 5400 اصل کمتر باشه ) یا میزان رم و xxup xxunk سیستم و بافر آن یا حتی xxup cpu که در مچ بودن سیستم و انتقال سرعت بسیار مهمه و از همه مهمتر نوع پورت xxup usb سیستم و جنس """,recommended
1,xxbos چای سفید با کیفیت اصیل \r\n وقتی برای اولین مرتبه یک محصولی همانند این محصول که چای سفید هست میخواهیم امتحان کنیم ، معمولا در اینترنت جستجویی انجام xxunk و اطلاعات کلی در مورد محصول کسب میکنیم . قبل از خرید این محصول من هم یک تحقیق کلی در زمینه چای سفید xxunk از شکل ظاهری ، طعم و عطر محصول انجام xxunk از رسیدن سفارش و باز نمودن بسته اولین موردی که سبب xxunk xxunk مطابقت شکل ظاهری برگه های چای سفید این محصول با عرف استاندارد برگه های چای سفید بود . شاید بگید مگر می شود فرق محسوس و زیادی داشته باشد در پاسخ باید بگویم متاسفانه یک برند مشهور دیگر را در این زمینه نیز امتحان نمودم و محتوی داخل بسته بندی به تنها چیزی که شباهت نداشت چای سفید بود ! خب از توضیحات این محصول و تجربه ای که داشتم خارج نشیم ،,recommended
2,xxbos از انتخابم کاملا راضیم \r\n من ازش راضیم خودم اسفند 95 خریدمش واقعا دوست داشتنیه . بارها پیش اومده که xxunk یه کاری میکردیم لپ تاب اپل با سیستم عامل مکینتاش نکشیده و xxunk کرده ولی لپ تاب من عین بنز انجام داده . البته قبول دارم که به خود کاربر هم بستگی داره که کامپیوتر رو نزنه داغون کنه و xxunk اش به xxunk ها باشه و … ولی واقعا خوبه . اگه هم مشکلی بوده به خاطر باگ xxunk آپدیت های ویندوز بوده من از خودش مشکلی ندیدم . فارسی نداشتن رو کیبورد رو خیلی ها نقطه ضعف نوشتن واقعا کارش فقط یه برچسبه نمی دونم نقطه ضعفش چیه خب واسه ایران xxunk که نقطه ضعف نیست . در مورد پورت هاش عالیه یه وقتایی باید چند تا فلش بزنم و همزمان دانگل موس هم هست ( موس اش عالیه ولی من دوست دارم دستم کنار کامپیوتر,recommended
3,xxbos xxunk \r\n با سلام . بنده بعد از کلی xxunk هفته پیش این دوربین رو خریدم . xxunk تخصصی این دوربین ارزش خریدن داره و بسیار کاربردی هست . چون کیفیت تصویر نسبت به سایر دوربینها بسیار خوبه . دید در شب بسیار عالی . تشخیص حرکت و آلارم این دوربین واقعا خوبه و قابلیت اتصال به دستگاههای جانبی نیز داره . فقط برای اتصال به وای فای کمی کند هست و اگه برق بره برای وصل شدن مجدد خیلی کند هست بهترین حالت اینه که بصورت کابل به مودم وصل بشه . مشکل دیگه اینکه با شبکه همراه موبایل بعضی وقتا کار نمیده که اونم مربوط به اینترنت مسخره ایران هست.اگر سرعت اینترنت تون زیر 19 kb / s باشه عملا هیچ دستوری رد و بدل xxunk . عمده ترین مشکل اینه که بدون اینترنت کاربردش خیلی محدود میشه . احتمالا اگه نزدیک دوربین باشین میتونید xxunk کنید,recommended
4,xxbos استفاده ی دوساله \r\n امروز میشه دقیقا دوساله که دارم از این پروژکتور استفاده میکنم ، دو ماه بعد از اینکه اینو خریدم مدل mp - xxunk به بازار اومد ولی بازم از خریدم راضیم چون واقعا بهترین xxunk که تو عمرم xxunk نتیجه استفاده ای که با این پروژکتور داشتم باید بگم که کیفیت تصویر بس xxrep 4 ی ار واضحی داره ولی بدونید که اگر میخواید بهترین کیفیت رو دریافت کنید باید چراغ هارو خاموش کنید و شب هم باشه ( من که خودم هر موقع بخوام فیلم نگاه کنم چه تو این پروژکتور یا تلویزیون xxunk خاموش میکنم چون به آدم حس بهتری میده بخصوص با این xxunk ) ، فقط یه توصیه مهم بهتون بکنم ، با خرید این ، یک بلند گوی کوچیک که جایگاه ورودی aux داشته باشه رو هم خریداری کنید چون بلندگوی خودش فقط برای دو الی سه نفر پاسخگو هست,recommended
5,xxbos مناسب فیلم دیدن \r\n حدود ۶ ماه هست که این پروژکتور را خریدم ، یک پرده نمایش xxunk متر هم همزمان از دیجی با قیمت عالی خریدم .. و داخل اتاقم نصب xxunk سینما را داخل خونه آوردم . در مورد خود دستگاه که با توجه به قیمت عالیه،فقط کافیه یه فیلم 1080 دانلود و از گوشی پخش کنید تا به کیفیت این پروژکتور پی xxunk بار استوانه ای یک متری هم پایین پرده نصب کردم ( که اونم از دیجی خان خریدم ) . گوشی نوت ۸ را از طریق xxup hdmi ب

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.618544,0.613677,0.702524,01:09


epoch,train_loss,valid_loss,accuracy,time
0,0.616603,0.607997,0.702524,01:43
1,0.613468,0.607608,0.702524,01:42


In [ ]:
learn.show_results()

,text,category,category_
0,xxbos بررسی هدست ( گردنی ! ) 1 \r\n اولی که دیدمش با یه جعبه ی مشکی که از چند نقطه پاره بود مواجه شدم . درون جعبه یه قالب پلاستیکی ( نه چندان تمییز ) که هدست توش قرار داشت ، دو جفت سریِ قابل xxunk ، راهنمای مصرف کننده و یه کابل شارژ میکرو یو اس xxunk خیلی کوتاه موجود بود . \r\n این محصول با نام xxup s9 متعلق به شرکت xxup xxunk و ساخت xxunk چینه که در چهار رنگِ قرمز ، نقره ای ، طلایی و زرد به بازار عرضه شده . روی جعبه برچسب اصالت شرکت هم زده شده . \r\n نکته ای که از اول برام عجیب اومد تفاوت گفته های روی جعبه با برگه ی راهنمای استفاده کننده بود . که اطلاعات روی برگه ی راهنما به واقعیت xxunk . \r\n روی جعبه زمان دشارژ شدن باتری در حالت استفاده ی مداوم,recommended,recommended
1,xxbos تجربه دو ماه استفاده \r\n یک مدل قدیمی ریش تراش فیلیپس داشتم که از کار افتاد اومدم توی دیجی کالا و چند روز تمام مدل های ریش تراش را بالا پایین کردم و نظرات را خوندم و عملا هم نظرات کمک شایانی به انتخاب شما نمیکند چون یکی صددرصد تایید میکنه و یکی اصلا بهتون پیشنهاد نمیده ! در نهایت دل رو زدم به دریا و این مدل را خریدم هرچند که قیمت به نسبت بالایی داشت . وقتی به دستم رسید و بازش کردم اول چیزی که جلب توجه کرد ارگونومی خوب و خوش دست بودنشه که باعث میشه خوب توی دستتون قرار بگیره جایی هم که xxunk شما قرار میگیره از جنس لاستیک هست که به دست شما میچسبه و نگرانی شما را از افتادنش در حین اصلاح کمتر میکنه . رنگ و طرح جذابش هم بهتون حس خوشایندی میده . سر سه تایی اون هم از,recommended,recommended
2,xxbos نقد پس از ۳ هفته استفاده \r\n با سلام \r\n بجز سایز صفحه نمایش ، مشخصات و عملکرد مدل ۱۰ و ۹ اینچی کاملا یکسانه بنابراین چنانچه در خرید بین این دو مدل مردد هستید با خواندن این بررسی میتونید تا حدودی زیادی تصمیم بگیرید . باتری هر دو مدل 2200 میلی آمپری و رزولوشن صفحه هر دو xxunk در 600 پیکسل هست که تراکم پیکسلی 132 برای مدل ۹ اینچ و 110 رو برای مدل 10 اینچ رقم میزنه . با توجه به این دو مورد بنظر من خرید مدل ۹ اینچ منطقی تره با اینحال سلیقه هر کس متفاوته . \r\n اگر تابحال محصولات مارشال رو تجربه کرده باشید قطعا میدونید که همگی اونها یک خصوصیت مشترک دارند : ارائه امکانات فراوان در کنار قیمت رقابتی و دارا بودن کیفیت معمولی . این مانیتور هم از این xxunk xxunk نیست و در کنار آپشن های متفاوتی که,recommended,recommended
3,xxbos برسی کیف xxunk xxup xxunk \r\n من این کیف به قصد دوتا کیف گرفتم ، در واقع خواستم هم جای یک مودم همراه به اندازه xxunk cm در xxunk cm به ( عمق یا ارتفاع 1.5 cm ) و هم جای یک هارد وسترن به ( عمق 1.9 cm ) باشه اما متاسفانه جواب نداد ! اشکال کار هم در اندازه مشخص شده برای عمق کیف در عکس بود … که زده شده 4.4 cm اما گفته نشده که اندازه بیرونی کیف محاسبه شده است ! ! و من هم به خاطر همین اندازه به اشتباه افتادم و خریدم انجام دادم ، حالا فقط میتونم یا هارد با عمق نهایتا 2.5 cm بزارم یا مودم تا این اندازه عمق + اشیای کوچک مثل شارژر + کابل + فلش + رم ریدر و … \r\n اندازه های واقعی داخل کیف : ( نه از جداره ایی که در عکس مشخص,recommended,recommended
4,xxbos قابل قبول \r\n با سلام . ابتدا باید از شرکت xxunk سرویس بابت پیگیری جدی و مستمر در مورد نصب دستگاه تشکر کنم متاسفانه مسئول نصب با اینکه از تلویزیون وستل تعریف می کرد اما اطلاعات چندانی در مورد نصب این تلویزیون چه بخش سخت افزار و چه بخش نرم افزار نداشت به محض راه اندازی شاهد تصویر بسیار ضعیف و ناامید کننده و استفاده از اندرویدی کاملا محدود بودم . دو کنترل در کارتن بود که یکی از اونها کار نمی کرد . بعد از رفتن نصاب دفترچه رو خوندم و چند ساعتی درگیر تنظیمات شدم که نتیجه اون رو خدمت خریداران احتمالی عرض می کنم : \r\n 1- گیرندگی تلویزیون دیجیتال قوی نیست و حتما از یک آنتن خوب استفاده کنید در این صورت بر خلاف توضیحات دیجی کالا که کیفیت تصویر رو معمولی ذکر کرده شاهد تصویری نزدیک به عالی و فراتر از حد انتظار خواهید,recommended,recommended
5,xxbos کیفیت ساخت عالی \r\n من این محصول را یک سال پیش خریده بودم و هنوز هم کار میکند و واقعا از همه نظر عالی است بجز اینکه دکمه های back / forward کمی کوچک طراحی شده اند . \r\n موسی که من تجربه ی خوبی با آن داشتم دقیقا همین مدل بود فقط گیرنده اش 5 g بود ولی این دومی که امروز خریده ام گیرنده اش 2.4 g است . دقیقا مطمین نیستم افت کیفیت داشته باشد ( امیدوارم مثل سری اول خوش ساخت و با کیفیت و دقیق باشه ) ولی کلا جهت اطلاع مصرف کننده ها میگم که 5 g قدرت سیگنال

In [ ]:
learn.predict("خیلی عالی بود")

('recommended', tensor(1), tensor([0.2596, 0.7404]))

### Fine-Tune pretrained Language Model on our data

In [ ]:
dls_lm = TextDataLoaders.from_df(df, text_col="text", label_col="recommend", is_lm=True, valid_pct=0.1)
dls_lm.show_batch(max_n=5)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,xxbos کارایی بسیار کم \r\n برخلاف توضیحات دوربین نداره و انتن دهی بسیار ضعیفی xxunk با مدلهای ارزونتر نداره xxbos xxunk \r\n افتضاح بود واقعا توصیه نمیکنم xxbos فوق العاده \r\n کیفیت درجه یک وعالی xxbos xxunk \r\n من این گوشی رو برای پسرم خریدم ازش راضی ام xxbos چرخ موزیکال \r\n کیفیت چندان نداره با توجه به xxunk سایزش از چیزی که در مشخصات نوشته شده کوچکتره . xxbos قابی بی,کارایی بسیار کم \r\n برخلاف توضیحات دوربین نداره و انتن دهی بسیار ضعیفی xxunk با مدلهای ارزونتر نداره xxbos xxunk \r\n افتضاح بود واقعا توصیه نمیکنم xxbos فوق العاده \r\n کیفیت درجه یک وعالی xxbos xxunk \r\n من این گوشی رو برای پسرم خریدم ازش راضی ام xxbos چرخ موزیکال \r\n کیفیت چندان نداره با توجه به xxunk سایزش از چیزی که در مشخصات نوشته شده کوچکتره . xxbos قابی بی کیفیت
1,بعضی اوقات اگه جسمی بین گوشی و اسپیکر قرار بگیره قطع و وصل میشه . \r\n صدای اسپیکر واقعا خوب و با کیفیته حتی صدای اون خانم که آزار دهنده نیست ( در حد xxunk و میوزیک ) \r\n▁ تولید xxup bass مناسبی داره و برای همه جور آهنگی بدون مشکله . \r\n توی بعضی آهنگها صدای خفگی ازش میاد ولی بازهم با کیفیته و من راضیم ازش . \r\n توی صدای,اوقات اگه جسمی بین گوشی و اسپیکر قرار بگیره قطع و وصل میشه . \r\n صدای اسپیکر واقعا خوب و با کیفیته حتی صدای اون خانم که آزار دهنده نیست ( در حد xxunk و میوزیک ) \r\n▁ تولید xxup bass مناسبی داره و برای همه جور آهنگی بدون مشکله . \r\n توی بعضی آهنگها صدای خفگی ازش میاد ولی بازهم با کیفیته و من راضیم ازش . \r\n توی صدای بلند
2,.. xxunk کشور چین xxunk بده که حداقل وسیله ای xxunk قابل استفاده است .. xxbos تو پیشنهاد ویژه خریدم وعالی بود \r\n کیف خوبی هست و تو پیشنهاد ویژه ارزش خرید دارد xxbos لباس کودک \r\n خوبه،فقط شلوار انگار ی سایز کوچکتره نسبت به بلوز xxbos ساعت صفحه آبی عالیه \r\n ساعت فقط در چین مونتاژ شده و موتور ژاپنیه . کیفیت هم برخلاف نظر برخی دوستان بالاست و زیبایی خاصی,xxunk کشور چین xxunk بده که حداقل وسیله ای xxunk قابل استفاده است .. xxbos تو پیشنهاد ویژه خریدم وعالی بود \r\n کیف خوبی هست و تو پیشنهاد ویژه ارزش خرید دارد xxbos لباس کودک \r\n خوبه،فقط شلوار انگار ی سایز کوچکتره نسبت به بلوز xxbos ساعت صفحه آبی عالیه \r\n ساعت فقط در چین مونتاژ شده و موتور ژاپنیه . کیفیت هم برخلاف نظر برخی دوستان بالاست و زیبایی خاصی هم
3,و مرجوع کردم که دیگر موجود نبود بار دوم برند xxup xxunk که متاسفانه این یکی هم که امروز بدستم رسید قطعات فلزی و بست شیشه چراغ ندارد xxunk است که باید آنها را از چراغ قدیمی باز کنم و روی چراغ جدید ببندم . xxbos از همه لحاظ عالی هستش \r\n اولش که خریدم ترسیدم چون خیلی از دوستان نوشته بودن که پاک نمیشه و بی کیفیت هست ولی اصلا اینطور,مرجوع کردم که دیگر موجود نبود بار دوم برند xxup xxunk که متاسفانه این یکی هم که امروز بدستم رسید قطعات فلزی و بست شیشه چراغ ندارد xxunk است که باید آنها را از چراغ قدیمی باز کنم و روی چراغ جدید ببندم . xxbos از همه لحاظ عالی هستش \r\n اولش که خریدم ترسیدم چون خیلی از دوستان نوشته بودن که پاک نمیشه و بی کیفیت هست ولی اصلا اینطور نیست
4,… \r\n البته کسایی که میگن با در اوردن نادرست خراب میشه .. یه بار برق ما رفت و هیچ اتفاقی نیافتاد .. xxunk خود کنسول گفت که احتمال داره آسیب دیده باشه ) ولی من هیچ مشکلی ندیدم . xxbos گرونه \r\n با این قیمت واقعاً نمیارزه ! xxbos نوری \r\n خیلی نازک و شله و فیت گوشی هم نیست . من یه کاور با نصف قیمت این خریدم خیلی از,\r\n البته کسایی که میگن با در اوردن نادرست خراب میشه .. یه بار برق ما رفت و هیچ اتفاقی نیافتاد .. xxunk خود کنسول گفت که احتمال داره آسیب دیده باشه ) ولی من هیچ مشکلی ندیدم . xxbos گرونه \r\n با این قیمت واقعاً نمیارزه ! xxbos نوری \r\n خیلی نازک و شله و فیت گوشی هم نیست . من یه کاور با نصف قیمت این خریدم خیلی از این


In [ ]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], wd=0.1).to_fp16()

In [ ]:
# fit last layer
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.859764,5.564551,0.150809,261.008087,01:33
1,5.411556,5.245706,0.175284,189.749741,01:32


In [ ]:
# fit hidden layers
learn.unfreeze()
learn.fit_one_cycle(2, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.986550,4.879456,0.208773,131.559021,01:34
1,4.803213,4.777107,0.217326,118.760307,01:35


In [ ]:
TEXT = "گوشی خوبی هست ولی"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]
print("\n".join(preds))

گوشی خوبی هست ولی خیلی خوبه ، کیفیت این مدل فوق العاده هست 
 واسه شارژ کردن گوشی عالیه به راحتی شارژ نمیشه و کثیف میشه از خریدش پشیمون نمیشید 
 من این محصول رو از دیجی کالا خریدم و خیلی راضیم و
گوشی خوبی هست ولی من با توجه به قیمت ی ویژه خریدم و راضیم 
 من که باهاش ازش استفاده میکنم و کاملا راضی هستم . حامدزاده 
 بسیار عالی و خوب و زیبا بهترین برند 
 هندزفری خوبیه بهترین گزینه برای


In [ ]:
learn.save_encoder('fine_tuned')

### Use new language model for text classification task

In [ ]:
dls_clas = TextDataLoaders.from_df(df, text_col="text", 
                                   label_col="recommend", 
                                   valid_pct=0.1, 
                                   text_vocab=dls_lm.vocab)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn = learn.load_encoder('fine_tuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.254043,0.189401,0.926149,01:09


In [ ]:
learn.predict("خیلی بد بود")

('not_recommended', tensor(0), tensor([0.9073, 0.0927]))

In [ ]:
learn.predict("افتضاح بود")

('not_recommended', tensor(0), tensor([0.9504, 0.0496]))

In [ ]:
learn.predict("عالی بود")

('recommended', tensor(1), tensor([0.0309, 0.9691]))

In [ ]:
learn.predict("استفاده از این محصول خیلی باعث رضایت من شد")

('recommended', tensor(1), tensor([0.2770, 0.7230]))